In [1]:
import os

import flwr as fl
import tensorflow as tf

/Users/guilhermeborges/miniconda3/envs/trctf/lib/python3.9/site-packages/jax/_src/lib/__init__.py:34: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Load model and data (MobileNetV2, CIFAR-10)
model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

Metal device set to: Apple M1


In [3]:
# Define Flower client
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        #print(f"[Client {self}] config")
        #print(model.get_weights())
        print("testeeee")
        
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        model.fit(x_train, y_train, epochs=1, batch_size=32)
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        
        model.set_weights(parameters)
        # Evaluate global model parameters on the local test data
        loss, accuracy = model.evaluate(x_test, y_test)
        #loss, accuracy = model.evaluate(x_test, y_test)
        print("test metrics: ",accuracy)
        
        # Return results, including the custom accuracy metric
        num_examples_test = len(x_test)
        return loss, num_examples_test, {"accuracy": accuracy}

In [4]:
# Start Flower client
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=CifarClient())

INFO flwr 2023-07-19 09:58:19,794 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-07-19 09:58:19,798 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-07-19 09:58:19,806 | connection.py:39 | ChannelConnectivity.READY


testeeee


DEBUG flwr 2023-07-19 10:01:05,489 | connection.py:113 | gRPC channel closed


KeyboardInterrupt: 